In [5]:
#Title 

In [6]:
#Deliverable 1 Population Statistics

In [7]:
!pip install wbdata

In [8]:
import wbdata
wbdata.get_topics()

Key '1635257009201650005' not in persistent cache.
Key '4446260396718935972' not in persistent cache.
Key '2660095863787826343' not in persistent cache.
Key '-1574930037329744515' not in persistent cache.
Key '-866863910299146213' not in persistent cache.
Key '1944139477218139754' not in persistent cache.
Key '-6217247699421755418' not in persistent cache.
Key '1445941785561171016' not in persistent cache.
Key '-2698178184873871149' not in persistent cache.
Key '-7064929164908614826' not in persistent cache.
Key '3181523581211977156' not in persistent cache.
Key '-1739794454289542773' not in persistent cache.
Key '-7084700360408538847' not in persistent cache.
Key '578489124574387587' not in persistent cache.
Key '-3565630845860654578' not in persistent cache.
Key '4097558639122271856' not in persistent cache.
Key '4260890120989069126' not in persistent cache.
Key '-3900497055650826046' not in persistent cache.
Key '-7590788311604595872' not in persistent cache.
Key '-38142754110294177

  id  value
----  -------------------------------
   1  Agriculture & Rural Development
   2  Aid Effectiveness
   3  Economy & Growth
   4  Education
   5  Energy & Mining
   6  Environment
   7  Financial Sector
   8  Health
   9  Infrastructure
  10  Social Protection & Labor
  11  Poverty
  12  Private Sector
  13  Public Sector
  14  Science & Technology
  15  Social Development
  16  Urban Development
  17  Gender
  18  Millenium development goals
  19  Climate Change
  20  External Debt
  21  Trade

In [9]:
import pandas as pd
import numpy as np

In [10]:
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Glob

In [11]:
indicators = wbdata.get_indicators(source=40)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

In [12]:
sexdict = {'Male':'MA', 'Female':'FE'}
def population(year,sex,age_range,place):
    lower, upper = age_range[0], age_range[1]
    assert upper >= lower, "Invalid date range!" #Part of Deliverable 2
    assert sex in ["Male", "Female", "Both"], "Invalid sex Please use one of the following: Male, Female, Both"
    if sex == "Both":
        return population(year,"Male",age_range,place) + population(year,"Female",age_range,place)
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df = df[df['date'] == f'{year}-01-01'] # selecting relevant year
        df.drop('date', axis=1, inplace=True)
        extra = ((yrs[-1][1] - upper) / 5) * (df.iloc[:,-1].to_list()[0]) # controls for age ranges not on bounds, could change to 
                                                                          # year by year subtraction)
    sum = df.sum(axis=1).iloc[0]
    return (sum - extra)

In [13]:
population(1990, 'Male', (10,33), 'CHN')

np.float64(263801536.6)

In [14]:
#Deliverable 3 Population Data Frames

In [15]:
def population_data_frames(year, sex, age_range, place):
    lower, upper = age_range[0], age_range[1]
    assert upper >= lower, "Invalid date range!" #Part of Deliverable 2
    assert sex in ["Male", "Female", "Both"], "Invalid sex Please use one of the following: Male, Female, Both"
    if sex == "Both":
        return population_data_frames(year,"Male",age_range,place).merge(population_data_frames(year,"Female",age_range,place), on="date")
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df['country'] = place
        df.set_index(['country', 'date'], inplace=True)
    return df

In [16]:
#population_data_frames_for_males

In [17]:
population_data_frames_for_males_age_10_to_33 = population_data_frames(1990, 'Male', (10,33), 'CHN')
population_data_frames_for_males_age_10_to_33.rename(columns={'Male 1014': 'Male ages 10-14', 
                                                 'Male 1519': 'Male ages 15-19', 
                                                 'Male 2024': 'Male ages 20-24', 
                                                 'Male 2529': 'Male ages 25-29',
                                                 'Male 3034': 'Male ages 30-34'},
                                                 inplace=True)
population_data_frames_for_males_age_10_to_33

Male ages 10-14  Male ages 15-19  Male ages 20-24  \
country date                                                            
CHN     2023-01-01       47697475.0       43619845.0       42690196.0   
        2022-01-01       46945694.0       42880142.0       43079433.0   
        2021-01-01       46077575.0       42441714.0       43637348.0   
        2020-01-01       45390757.0       42444894.0       44208685.0   
        2019-01-01       44534610.0       42626158.0       45014490.0   
...                             ...              ...              ...   
        1964-01-01       45210547.0       32772310.0       27179605.0   
        1963-01-01       43265071.0       31245247.0       27269359.0   
        1962-01-01       40877563.0       29902774.0       27482213.0   
        1961-01-01       38596810.0       29125584.0       27700640.0   
        1960-01-01       36823111.0       29027168.0       28092415.0   

                    Male ages 25-29  Male ages 30-34  
country date                                          
CHN     2023-01-01       45996968.0       58571312.0  
        2022-01-01       47322938.0       61347262.0  
        2021-01-01       48943931.0       63431979.0  
        2020-01-01       51917714.0       63016755.0  
        2019-01-01       55706672.0       60783908.0  
...                             ...              ...  
        1964-01-01       26566937.0       24330081.0  
        1963-01-01       26878889.0       23871048.0  
        1962-01-01       26824171.0       23384228.0  
        1961-01-01       26562875.0       23028989.0  
        1960-01-01       26459425.0       22874774.0  

[64 rows x 5 columns]

In [18]:
df = pd.DataFrame({'old_name': [1, 2], 'another_old_name': [3, 4]})

# Rename specific columns using a dictionary
df.rename(columns={'old_name': 'new_name', 'another_old_name': 'another_new_name'}, inplace=True)

# Rename all columns by assigning a new list of names
df.columns = ['new_name1', 'new_name2']

In [19]:
population_data_frames(1990, 'Both', (10,33), 'CHN')

,Male 1014,Male 1519,Male 2024,Male 2529,Male 3034,Female 1014,Female 1519,Female 2024,Female 2529,Female 3034
date,,,,,,,,,,
2023-01-01,47697475.0,43619845.0,42690196.0,45996968.0,58571312.0,40892243.0,37182969.0,36605841.0,40184117.0,52861422.0
2022-01-01,46945694.0,42880142.0,43079433.0,47322938.0,61347262.0,40161573.0,36559731.0,37035577.0,41562725.0,55702933.0
2021-01-01,46077575.0,42441714.0,43637348.0,48943931.0,63431979.0,39352043.0,36210584.0,37629021.0,43238213.0,57929324.0
2020-01-01,45390757.0,42444894.0,44208685.0,51917714.0,63016755.0,38718791.0,36255224.0,38255106.0,46168527.0,57874577.0
2019-01-01,44534610.0,42626158.0,45014490.0,55706672.0,60783908.0,37959894.0,36469477.0,39112543.0,49873080.0,56135406.0
...,...,...,...,...,...,...,...,...,...,...
1964-01-01,45210547.0,32772310.0,27179605.0,26566937.0,24330081.0,43236900.0,30062519.0,24208269.0,23499927.0,22007969.0
1963-01-01,43265071.0,31245247.0,27269359.0,26878889.0,23871048.0,41174542.0,28433763.0,24118305.0,23923846.0,21660173.0
1962-01-01,40877563.0,29902774.0,27482213.0,26824171.0,23384228.0,38550727.0,27084802.0,24120873.0,24034501.0,21287773.0


In [20]:
#Deliverable 2 Unit Tests

In [28]:
#assert population(year=2000,sex='Male',age_range=(0,100),place='WLD') > 0.0000000000000000001,"Too few males!"
population_data_frames(year=2000,sex='Male',age_range=(0,31),place='WLD')



population_data_frames_for_males_ages_0_to_31 = population_data_frames(1990, 'Male', (10,33), 'CHN')
population_data_frames_for_males_ages_0_to_31.rename(columns={'Male 0004': 'Male ages 00-04',
                                                              'Male 0509': 'Male ages 05-09',
                                                              'Male 1014': 'Male ages 10-14',
                                                              'Male 1519': 'Male ages 15-19',
                                                              'Male 2024': 'Male ages 20-24',
                                                              'Male 2529': 'Male ages 25-29',
                                                              'Male 3034': 'Male ages 30-34',},
                                                              inplace=True)
population_data_frames_for_males_ages_0_to_31

Male ages 10-14  Male ages 15-19  Male ages 20-24  \
country date                                                            
CHN     2023-01-01       47697475.0       43619845.0       42690196.0   
        2022-01-01       46945694.0       42880142.0       43079433.0   
        2021-01-01       46077575.0       42441714.0       43637348.0   
        2020-01-01       45390757.0       42444894.0       44208685.0   
        2019-01-01       44534610.0       42626158.0       45014490.0   
...                             ...              ...              ...   
        1964-01-01       45210547.0       32772310.0       27179605.0   
        1963-01-01       43265071.0       31245247.0       27269359.0   
        1962-01-01       40877563.0       29902774.0       27482213.0   
        1961-01-01       38596810.0       29125584.0       27700640.0   
        1960-01-01       36823111.0       29027168.0       28092415.0   

                    Male ages 25-29  Male ages 30-34  
country date                                          
CHN     2023-01-01       45996968.0       58571312.0  
        2022-01-01       47322938.0       61347262.0  
        2021-01-01       48943931.0       63431979.0  
        2020-01-01       51917714.0       63016755.0  
        2019-01-01       55706672.0       60783908.0  
...                             ...              ...  
        1964-01-01       26566937.0       24330081.0  
        1963-01-01       26878889.0       23871048.0  
        1962-01-01       26824171.0       23384228.0  
        1961-01-01       26562875.0       23028989.0  
        1960-01-01       26459425.0       22874774.0  

[64 rows x 5 columns]

In [22]:
df = pd.DataFrame({'old_name': [1, 2], 'another_old_name': [3, 4]})

# Rename specific columns using a dictionary
df.rename(columns={'old_name': 'new_name', 'another_old_name': 'another_new_name'}, inplace=True)

# Rename all columns by assigning a new list of names
df.columns = ['new_name1', 'new_name2']

In [25]:
population_data_frames_for_both_ages_10_to_33 = population_data_frames(1990, 'Both', (10,33), 'CHN')
population_data_frames_for_both_ages_10_to_33.rename(columns={'Male 1014': 'Male ages 10-14', 
                                                              'Male 1519': 'Male ages 15-19',
                                                              'Male 2024': 'Male ages 20-24',
                                                              'Male 2529': 'Male ages 25-29',
                                                              'Male 3024': 'Male ages 30-24',
                                                              'Female 1014': 'Female ages 10-14',
                                                              'Female 1519': 'Female ages 15-19',
                                                              'Female 2024': 'Female ages 20-24',
                                                              'Female 2529': 'Female ages 25-29',},
                                                              inplace=True)
population_data_frames_for_both_ages_10_to_33

,Male ages 10-14,Male ages 15-19,Male ages 20-24,Male ages 25-29,Male 3034,Female ages 10-14,Female ages 15-19,Female ages 20-24,Female ages 25-29,Female 3034
date,,,,,,,,,,
2023-01-01,47697475.0,43619845.0,42690196.0,45996968.0,58571312.0,40892243.0,37182969.0,36605841.0,40184117.0,52861422.0
2022-01-01,46945694.0,42880142.0,43079433.0,47322938.0,61347262.0,40161573.0,36559731.0,37035577.0,41562725.0,55702933.0
2021-01-01,46077575.0,42441714.0,43637348.0,48943931.0,63431979.0,39352043.0,36210584.0,37629021.0,43238213.0,57929324.0
2020-01-01,45390757.0,42444894.0,44208685.0,51917714.0,63016755.0,38718791.0,36255224.0,38255106.0,46168527.0,57874577.0
2019-01-01,44534610.0,42626158.0,45014490.0,55706672.0,60783908.0,37959894.0,36469477.0,39112543.0,49873080.0,56135406.0
...,...,...,...,...,...,...,...,...,...,...
1964-01-01,45210547.0,32772310.0,27179605.0,26566937.0,24330081.0,43236900.0,30062519.0,24208269.0,23499927.0,22007969.0
1963-01-01,43265071.0,31245247.0,27269359.0,26878889.0,23871048.0,41174542.0,28433763.0,24118305.0,23923846.0,21660173.0
1962-01-01,40877563.0,29902774.0,27482213.0,26824171.0,23384228.0,38550727.0,27084802.0,24120873.0,24034501.0,21287773.0
